In [ ]:
import pandas as pd
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

url = 'https://raw.githubusercontent.com/Alcalyxz/datasetProjectIA/main/Training-Dataset.csv'

df = pd.read_csv(url)
print(df.head(5))

   having_IP_Address  URL_Length  ...  Statistical_report  Result
0                 -1           1  ...                  -1      -1
1                  1           1  ...                   1      -1
2                  1           0  ...                  -1      -1
3                  1           0  ...                   1      -1
4                  1           0  ...                   1       1

[5 rows x 31 columns]


In [ ]:
df = df.drop(['having_Sub_Domain'], axis=1)
df = df.drop(['URL_of_Anchor'], axis=1)
df = df.drop(['Links_in_tags'], axis=1)
df = df.drop(['Redirect'], axis=1)
df = df.drop(['web_traffic'], axis=1)
df = df.drop(['Links_pointing_to_page'], axis=1)

##Debido a la alta correlaciòn (0.95) con PopUpWindow se elimina una de las dos ya que le aportan
##La misma informaciòn al sistema 
df = df.drop(['Favicon'], axis=1)

In [ ]:
df = df.drop(df[df['URL_Length']==0].index)
df = df.drop(df[df['SSLfinal_State']==0].index)
df = df.drop(df[df['SFH']==0].index)

In [ ]:
df = df.replace({-1: 0})

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
YEntrenamiento = df['Result']
XEntrenamiento = scaler.fit_transform(df.drop(['Result'],axis=1))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import itertools as itertools

In [ ]:
def experiementarSVC(x, y, kernels, gammas,params_reg):
    idx = 0
    kf = StratifiedKFold(n_splits=4)
    # crear una lista con la combinaciones de los elementos de cada list
    kernels_gammas_regs = list(itertools.product(kernels, gammas, params_reg))
    resultados = pd.DataFrame()
    
    for params in kernels_gammas_regs:
        kernel, gamma, param_reg = params
        print("parametros usados", params) 
        errores_train = []
        errores_test = []
        pct_support_vectors = []        
        for train_index, test_index in kf.split(x, y):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]  
            # normalizar los datos
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            svm = SVC(kernel=kernel, gamma=gamma, C=param_reg, max_iter = 100)
            # Entrenar el modelo
            svm.fit(X=X_train, y=y_train)
            # calculo de errores
            y_train_pred = svm.predict(X=X_train)
            y_test_pred = svm.predict(X=X_test)
            # error y pct de vectores de soporte
            errores_train.append(accuracy_score(y_true = y_train, y_pred =y_train_pred))
            errores_test.append(accuracy_score(y_true = y_test, y_pred = y_test_pred))
            # contar muestras de entrenamiento
            n_train = X_train.shape[0]
            pct_vs = (len(svm.support_vectors_)/n_train)*100
            pct_support_vectors.append(pct_vs)
        
        resultados.loc[idx,'kernel'] = kernel
        resultados.loc[idx,'gamma'] = gamma
        resultados.loc[idx,'param_reg'] = param_reg
        resultados.loc[idx,'error de entrenamiento'] = np.mean(errores_train)
        resultados.loc[idx,'error de prueba'] = np.mean(errores_test)
        resultados.loc[idx,'% de vectores de soporte'] = np.mean(pct_support_vectors)
        idx+=1
    return (resultados)

In [ ]:
resultadosSVC = experiementarSVC(x = XEntrenamiento,y=YEntrenamiento,kernels=['linear', 'rbf'],gammas = [0.01,0.1], params_reg = [0.001, 0.01,0.1, 1.0,10])
resultadosSVC

parametros usados ('linear', 0.01, 0.001)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('linear', 0.01, 0.01)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('linear', 0.01, 0.1)


...

parametros usados ('rbf', 0.1, 1.0)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


parametros usados ('rbf', 0.1, 10)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


,kernel,gamma,param_reg,error de entrenamiento,error de prueba,% de vectores de soporte
0,linear,0.01,0.001,0.629960,0.620396,2.930725
1,linear,0.01,0.010,0.648241,0.655787,2.930725
2,linear,0.01,0.100,0.628677,0.642815,2.923398
3,linear,0.01,1.000,0.600801,0.616544,2.835478
4,linear,0.01,10.000,0.562993,0.578193,2.696264
5,linear,0.10,0.001,0.629960,0.620396,2.930725
6,linear,0.10,0.010,0.648241,0.655787,2.930725
7,linear,0.10,0.100,0.628677,0.642815,2.923398
8,linear,0.10,1.000,0.600801,0.616544,2.835478
9,linear,0.10,10.000,0.562993,0.578193,2.696264
